In [1]:
import os
import csv
from process_data import pgn_to_dict, player_nationality
from chess_api import archived_games, monthly_games 
from settings import (USERNAME, CSV_DIR, GAME_RESULTS, COUNTRY_CODES, 
						ARCHIVE_ENDPOINT, NATIONALLITY_FILE, GAME_DATA_FILE)

In [2]:
def load_player_nationality():
	if NATIONALLITY_FILE in os.listdir(CSV_DIR):
		player_nat_dict = {}
		with open(os.path.join(CSV_DIR, NATIONALLITY_FILE), 'r') as player_nat:
			file_reader = csv.reader(player_nat,)
			for row in file_reader:
				player_nat_dict[row[0]] = row[1]
		return player_nat_dict
	else: return {}

In [3]:
def store_player_nationality(player_nationality: dict):
	with open(os.path.join(CSV_DIR, NATIONALLITY_FILE), 'w') as player_nat:
		file_writer = csv.writer(player_nat)
		for key, value in player_nationality.items():
			file_writer.writerow([key, value])

In [4]:
def store_game_data(game_data: dict, ):
	with open(os.path.join(CSV_DIR, GAME_DATA_FILE), 'w') as games:
		file_writer = csv.writer(games)
		headers = list(game_data.keys())
		file_writer.writerow(headers)
		for i in range(len(game_data[headers[0]])):
			row = []
			for header in headers:
				row.append(game_data[header][i])
			file_writer.writerow(row)

In [5]:
def sort_player_and_opponent(white: dict, black: dict):
	''' 
	Adds a new key 'side', to each dictonary
	returns a tuple (player, opponent) based on which had white and black
	'''
	white['side'] = 'White'
	black['side'] = 'Black'
	if white['username'].lower() == USERNAME.lower():
		return white, black		
	return black, white

In [6]:
def set_data_values(data: dict, player: dict, opponent: dict, pgn_dict: dict, 
					time_class, time_control, fen):
	''' '''
	data['Player'].append(player['username'])
	data['Player Rating'].append(player['rating'])
	data['Player Nationality'].append(player['nationality'])
	data['Player Side'].append(player['side'])
	data['Player Result'].append(GAME_RESULTS[player['result']])
	data['Player First Move'].append(pgn_dict['Moves'][1][player['side'][0]])
	
	data['Opponent'].append(opponent['username'])
	data['Opponent Rating'].append(opponent['rating'])
	data['Opponent Nationality'].append(opponent['nationality'])
	data['Opponent Side'].append(opponent['side'])
	data['Opponent Result'].append(GAME_RESULTS[opponent['result']])
	data['Opponent First Move'].append(pgn_dict['Moves'][1][opponent['side'][0]])

	data['Opening'].append(pgn_dict['ECOUrl'])
	data['Variation'].append(pgn_dict['Variation'])
	data['Termination'].append(pgn_dict['Termination'])
	data['Moves'].append(max(pgn_dict['Moves'].keys()))
	data['Date'].append(pgn_dict['EndDate'])
	data['Event'].append(pgn_dict['Event'])
	data['Tournament'].append(pgn_dict['Tournament'])
	data['Time Class'].append(time_class)
	data['Time Control'].append(time_control)
	data['FEN'].append(fen)

In [37]:
nationality_dict = load_player_nationality()
all_data ={
    'Player':[],'Player Rating':[],'Player Nationality':[],
    'Player Side':[],'Player Result':[],'Player First Move':[],
    'Opponent':[],'Opponent Rating':[],'Opponent Nationality':[],
    'Opponent Side':[],'Opponent Result':[],'Opponent First Move':[],
    'Opening':[],'Variation':[],'Termination':[],'Moves':[],'Date':[],
    'Tournament':[],'Event':[],'Time Class':[],'Time Control':[],'FEN':[],
}

for url in archived_games(ARCHIVE_ENDPOINT):
    for games in monthly_games(url):
        player, opponent = sort_player_and_opponent(games['white'], games['black'])
        player = player_nationality(COUNTRY_CODES, nationality_dict, player)
        opponent = player_nationality(COUNTRY_CODES, nationality_dict, opponent)
        if 'pgn' in games and 'ECOUrl' in games['pgn'] and 'rules' in games and games['rules'] == 'chess':
            pgn = pgn_to_dict(games['pgn'])
            if 'Tournament' not in pgn:
                pgn['Tournament'] = 'None'
            set_data_values(all_data, player, opponent, pgn, games['time_control'], 
                            games['time_class'], games['fen'])
store_game_data(all_data)
store_player_nationality(nationality_dict)

In [25]:
from chess_api import chess_com_data

tournaments_endpoint = 'https://api.chess.com/pub/player/dontmesswithme_2/tournaments'
data = chess_com_data(tournaments_endpoint)
for tournament in data['finished']:
    current_tournament_endpoint = tournament['@id']
    current_tournament_data = chess_com_data(current_tournament_endpoint)
    print('Tournament: {} -> settings: {}'.format(current_tournament_data['name'], current_tournament_data['settings']))

Tournament:  PNWCC Blitz - Jackpot III -> settings: {'type': 'swiss', 'rules': 'chess', 'is_rated': True, 'is_official': False, 'is_invite_only': False, 'user_advance_count': 1, 'winner_places': 3, 'registered_user_count': 174, 'total_rounds': 11, 'time_class': 'blitz', 'time_control': '1800+20'}
Tournament: *** US Chess 5|0 Blitz -> settings: {'type': 'swiss', 'rules': 'chess', 'is_rated': True, 'is_official': True, 'is_invite_only': False, 'user_advance_count': 1, 'winner_places': 3, 'registered_user_count': 45, 'total_rounds': 7, 'time_class': 'blitz', 'time_control': '3000+0'}
Tournament: 1|0 3 Check -> settings: {'type': 'swiss', 'rules': 'threecheck', 'is_rated': True, 'is_official': True, 'is_invite_only': False, 'user_advance_count': 1, 'winner_places': 3, 'registered_user_count': 26, 'total_rounds': 9, 'time_class': 'lightning', 'time_control': '600+0'}
Tournament: 1|0 3 Check -> settings: {'type': 'swiss', 'rules': 'threecheck', 'is_rated': True, 'is_official': True, 'is_invi

In [34]:
for url in archived_games(ARCHIVE_ENDPOINT):
    for games in monthly_games(url):
        
        if 'rules' in games and games['rules'] != 'chess':
            print(games['rules'])

kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
kingofthehill
crazyhouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
crazyhouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse
bughouse